# 2D Magnetic Loop Advection Test

This notebook runs a simulation of an advecting magnetic field loop with AsterX, and visualizes the data saved in OpenPMD format. 


### Test setup
In this test, a magnetized circular field loop is propagated within the surrounding non–magnetized ambient medium with
a constant velocity in a 2D periodic grid. The analytical prescription of magnetic field is:
- $B^x, B^y = -A_{loop} y/r, \ A_{loop} x/r; \ \ r < R_{loop} 
\\ \ \ \ \ \  \ \ \ \ \  \ = 0,  \ \ \ 0; \ \ r> R_{loop}$


- $A_{loop}$ is magnitude of magnetic field, $r$ is the cylindrical radius, $R_{loop}$ is the loop radius.

- Parameter settings: $\rho = 1.0$, $P_{gas} = 3.0$, $A_{loop} = 0.001$, $R_{loop} = 0.3$, $v^i = (1/12, 1/12, 1/24)$

- Here, we use the ideal gas EOS with $\gamma = 5/3$. 

- Initial data is set using the thorn ```AsterSeeds```.

- Grid domain is $[-0.5,+0.5]$ with size $40 \times 40$ (very low resolution).

- `Linear extrapolation` is used as boundary conditions in all directions.




In [ ]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)

# this (non-default package) keeps the end of shell output in view
try: import scrolldown
except ModuleNotFoundError: pass

# set Python environment 
import os, sys
os.environ["PYTHONUSERBASE"] = os.environ['HOME'] + "/Cactus/python"
sys.path.insert(1, f"{os.environ['PYTHONUSERBASE']}/lib/python{sys.version_info[0]}.{sys.version_info[1]}/site-packages")

## 1. Steps to perform the simulation

First, let's first move to the Cactus folder:

In [ ]:
cd ~/CarpetX

Now, let's create the parameter file to be used for this simulation:

In [ ]:
%%writefile ./par/Magnetic_loop_advection.par

ActiveThorns = "
    CarpetX
    Formaline
    IOUtil
    ODESolvers
    SystemTopology
    TimerReport
    ADMBase
    HydroBase
    TmunuBase
    AsterSeeds
    AsterX
    EOSX
"

$ncells = 40

# -------------------- Cactus --------------------------------------------------
Cactus::cctk_show_schedule = yes
Cactus::presync_mode       = "mixed-error"

Cactus::terminate       = "time"
Cactus::cctk_final_time = 10  


# -------------------- TimerReport ---------------------------------------------
TimerReport::out_every                  = 10
TimerReport::out_filename               = "TimerReport.asc"
TimerReport::output_all_timers_together = yes
TimerReport::output_all_timers_readable = yes
TimerReport::n_top_timers               = 50



# -------------------- CarpetX -------------------------------------------------
CarpetX::verbose = no

CarpetX::xmin = -0.5
CarpetX::ymin = -0.5
CarpetX::zmin = -1.0/$ncells  

CarpetX::xmax = +0.5
CarpetX::ymax = +0.5
CarpetX::zmax = +1.0/$ncells 

CarpetX::boundary_x = "none"
CarpetX::boundary_y = "none"
CarpetX::boundary_z = "linear extrapolation"

CarpetX::boundary_upper_x = "none"
CarpetX::boundary_upper_y = "none"
CarpetX::boundary_upper_z = "linear extrapolation"

CarpetX::periodic_x = yes
CarpetX::periodic_y = yes
CarpetX::periodic_z = no

CarpetX::ncells_x = $ncells
CarpetX::ncells_y = $ncells
CarpetX::ncells_z = 2

CarpetX::max_num_levels         = 1
CarpetX::regrid_every           = 100000
CarpetX::blocking_factor_x      = 1
CarpetX::blocking_factor_y      = 1
CarpetX::blocking_factor_z      = 1
CarpetX::regrid_error_threshold = 0.01

CarpetX::prolongation_type = "ddf"
CarpetX::ghost_size        = 3
CarpetX::dtfac             = 0.5


# -------------------- ODESolvers ----------------------------------------------
ODESolvers::method = "RK4"



# -------------------- ADMBase -------------------------------------------------
ADMBase::initial_data    = "Cartesian Minkowski"
ADMBase::initial_lapse   = "one"
ADMBase::initial_shift   = "zero"
ADMBase::initial_dtlapse = "none"
ADMBase::initial_dtshift = "none"


# -------------------- AsterSeeds ----------------------------------------------
AsterSeeds::test_type = "2DTest"
AsterSeeds::test_case = "magnetic loop advection"
AsterSeeds::mag_loop_adv_type      = "2D"
AsterSeeds::mag_loop_adv_axial_vel = "non-zero"

# -------------------- AsterX --------------------------------------------------
AsterX::reconstruction_method = "wenoz"
AsterX::flux_type = "HLLE"
AsterX::max_iter              = 100
AsterX::c2p_tol = 1e-8
AsterX::debug_mode = "yes"

Con2PrimFactory::unit_test = "yes"

EOSX::evol_eos_name = "IdealGas"
EOSX::gl_gamma = 1.66666666666667
EOSX::poly_gamma = 1.66666666666667

IO::out_dir = $parfile
IO::out_every =  20 
CarpetX::out_silo_vars = "
    HydroBase::rho
    HydroBase::Bvec
"

CarpetX::out_tsv_vars = "
    HydroBase::rho
    HydroBase::Bvec
"
        
CarpetX::out_openpmd_vars = "
    HydroBase::rho
    HydroBase::vel
    HydroBase::eps
    HydroBase::press
    HydroBase::Bvec
"


Then, submit the simulation using the following command:

In [ ]:
%%bash
./simfactory/bin/sim create-submit Magnetic_loop_advection \
  --parfile=./par/Magnetic_loop_advection.par --procs=3 --num-threads=1 --ppn-used=3 --walltime=00:20:00

The above command creates and runs the simulation ```Magnetic_loop_advection```, using the configuration ```sim```. The data is saved in the directory ```./simulations/Magnetic_loop_advection```.



In [ ]:
# watch log output, following along as new output is produced
!./simfactory/bin/sim show-output --follow Magnetic_loop_advection

## 2. Steps to visualize simulation data

The 2D data can be saved in both Silo format (which can be visualised, for instance, via VisIt) and in OpenPMD format. 

For further info on Silo, please visit: https://wci.llnl.gov/simulation/computer-codes/silo

For further info about OpenPMD, please visit:

- Official website:  https://www.openpmd.org
- GitHub repository: https://github.com/openPMD
- Documentation:     https://openpmd-api.readthedocs.io

Now, let's go back to the home directory:

In [ ]:
cd ~/

Import all the required modules:

In [ ]:
%pip install --user celluloid

In [ ]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from celluloid import Camera
import openpmd_api as io

Set the Matplotlib backend to `notebook`, not `inline`, since we'll want to animate some figures and the latter is not compatible with that

In [ ]:
%matplotlib notebook

Open a .bp file (ADIOS2 extension) as an OpenPMD **'series'**, which is a collection of **'iterations'**, each of which contains **'records'**, which are sets of either structured data --- **'meshes'** --- or unstructured data --- **'particles'**. 

AsterX only outputs mesh data. Each record has one or more **'components'**: for example, a record representing a scalar field only has one component, while a record representing a vector field has three.

In [ ]:
home = os.environ["HOME"]
series = io.Series(os.path.join(home, "simulations",
                                "Magnetic_loop_advection",
                                "output-0000","Magnetic_loop_advection",
                                "Magnetic_loop_advection.it%08T.bp5"), io.Access.read_only)

All iterations in our series have the same structure, i.e., they contain
the same records, since they all represent the same output, just at
different times. 

Here we define an empty Python nested dictionary whose
structure, once full, will be:

Iteration 0:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

Iteration 1:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

[...]

In [ ]:
iter_rec_comp_dict = {}

Print info, register data chunks and fill the above dictionary:

In [ ]:
for index in series.iterations:
    iteration = str(index)

    print("\nIteration " + iteration + ":")
    print("==============")

    # Allocate an empty dictionary associated to this iteration
    iter_rec_comp_dict[iteration] = {}

    i = series.iterations[index]

    for key in i.meshes:
        print("Components of record \"" + key + "\":")

        # Allocate an empty dictionary associated to this record. Notice that
        # 'record' is an OpenPMD mesh object, so it's better to use 'key'
        # instead of 'record' as a key in the dictionary ('record' could also be
        # used, but it makes accessing the key clumsy).
        record = i.meshes[key]
        iter_rec_comp_dict[iteration][key] = {}

        # Load each component of each record as a 'data chunk', i.e., an
        # allocated, but STILL INVALID, NumPy array. Later we will flush all
        # chunks (i.e., basically, fill the NumPy arrays) at once: this leads
        # to better I/O performance compared to flushing a large number of
        # small chunks. That's why we bothered creating the nested dictionary:
        # in this way, we can access the valid NumPy arrays for plotting
        # without having to flush each single chunk.
        # *IMPORTANT*: DO NOT access data chunks until flushing has happened!
        for component in record:
            print("    > " + component)  # 'component' is a string
            iter_rec_comp_dict[iteration][key][component] = record[component].load_chunk()  # *INVALID* 3D NumPy array

        print("")

Flush all registered data chunks, which are now **VALID** 3D NumPy arrays:

In [ ]:
series.flush()

Visualize a 2D movie of the x-component of the magnetic field on the xy plane:

In [ ]:
# Select the desired record and component to plot
record    = "hydrobase_bvec_lev00"  # "carpetx_regrid_error_lev00", "hydrobase_eps_lev00", "hydrobase_press_lev00", "hydrobase_rho_lev00", "hydrobase_vel_lev00"
component = "hydrobase_bvecx"  # "carpetx_regrid_error", "hydrobase_eps", "hydrobase_press", "hydrobase_rho", "hydrobase_velx", "hydrobase_vely", "hydrobase_velz"

# Set up the axes for the plot and the colorbar
fig    = plt.figure(figsize = [4.8, 4])
axplot = fig.add_axes([0.12, 0.14, 0.72, 0.74])
axclb  = fig.add_axes([0.85, 0.14, 0.02, 0.74])
#axplot = fig.add_axes([0.12, 0.14, 0.7, 0.7])
#axclb  = fig.add_axes([0.8, 0.16, 0.05, 0.65])


# Set title and labels
axplot.set_title("B^x", fontsize = 10., fontweight = "bold", color = "midnightblue")
axplot.set_xlabel("x", fontsize = 7.)
axplot.set_ylabel("z", fontsize = 7.)
axplot.tick_params(labelsize=7)
axplot.xaxis.set_major_locator(plt.MaxNLocator(5))
axplot.yaxis.set_major_locator(plt.MaxNLocator(5))


# Initialize the camera
camera = Camera(fig)

# Print frames
for iteration in iter_rec_comp_dict:
    # Retrieve the 3D array containing the data
    array3D = iter_rec_comp_dict[iteration][record][component]
    
    # Plot on the (x, y) plane at the half-way value of z
    # Notice that the 3D array is stored in (z, y, x) order
    z_index = int(array3D.shape[0]/2)
    x0     = np.linspace(-0.6, 0.6, array3D.shape[2])
    y0     = np.linspace(-0.6, 0.6, array3D.shape[1])
    image   = axplot.pcolormesh(x0, y0, array3D[z_index, :, :],
                                cmap = "RdBu", vmin = -0.001, vmax = 0.001)
    axplot.set_xlim(xmin=-0.5, xmax=0.5)
    axplot.set_ylim(ymin=-0.5, ymax=0.5)
    # Set up the colorbar
    axclb.tick_params(labelsize=7.0)
    fig.colorbar(image, cax = axclb, extend = "neither")
    
    # Print the current iteration
    axplot.text(0.18, 0.42, "Iteration " + iteration,
             fontsize = 8., fontweight = "bold", color = "k")

    # Take a snapshot of the figure at this iteration (needed later for the animation)
    camera.snap()

In [ ]:
from IPython.display import HTML
animation = camera.animate()
HTML(animation.to_html5_video())

You may delete the simulation directory via the following:


In [ ]:
%cd ~/
%rm -r ./simulations/Magnetic_loop_advection